In [4]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch import nn
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [10]:
# context initialization
with initialize(config_path="../training/conf/network/", job_name="test_app"):
    cfg = compose(config_name="vqvae")
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

encoder:
  _target_: text_recognizer.networks.vqvae.encoder.Encoder
  in_channels: 1
  hidden_dim: 32
  channels_multipliers:
  - 1
  - 2
  - 4
  - 4
  - 4
  dropout_rate: 0.25
decoder:
  _target_: text_recognizer.networks.vqvae.decoder.Decoder
  out_channels: 1
  hidden_dim: 32
  channels_multipliers:
  - 4
  - 4
  - 4
  - 2
  - 1
  dropout_rate: 0.25
_target_: text_recognizer.networks.vqvae.vqvae.VQVAE
hidden_dim: 128
embedding_dim: 32
num_embeddings: 1024
decay: 0.99

{'encoder': {'_target_': 'text_recognizer.networks.vqvae.encoder.Encoder', 'in_channels': 1, 'hidden_dim': 32, 'channels_multipliers': [1, 2, 4, 4, 4], 'dropout_rate': 0.25}, 'decoder': {'_target_': 'text_recognizer.networks.vqvae.decoder.Decoder', 'out_channels': 1, 'hidden_dim': 32, 'channels_multipliers': [4, 4, 4, 2, 1], 'dropout_rate': 0.25}, '_target_': 'text_recognizer.networks.vqvae.vqvae.VQVAE', 'hidden_dim': 128, 'embedding_dim': 32, 'num_embeddings': 1024, 'decay': 0.99}


In [11]:
net = instantiate(cfg)

In [12]:
from torchsummary import summary

In [13]:
summary(net, (1, 576, 640), device="cpu");

Layer (type:depth-idx)                   Output Shape              Param #
├─Encoder: 1-1                           [-1, 128, 18, 20]         --
|    └─Sequential: 2-1                   [-1, 128, 18, 20]         --
|    |    └─Conv2d: 3-1                  [-1, 32, 576, 640]        320
|    |    └─Conv2d: 3-2                  [-1, 32, 288, 320]        16,416
|    |    └─Mish: 3-3                    [-1, 32, 288, 320]        --
|    |    └─Conv2d: 3-4                  [-1, 64, 144, 160]        32,832
|    |    └─Mish: 3-5                    [-1, 64, 144, 160]        --
|    |    └─Conv2d: 3-6                  [-1, 128, 72, 80]         131,200
|    |    └─Mish: 3-7                    [-1, 128, 72, 80]         --
|    |    └─Conv2d: 3-8                  [-1, 128, 36, 40]         262,272
|    |    └─Mish: 3-9                    [-1, 128, 36, 40]         --
|    |    └─Conv2d: 3-10                 [-1, 128, 18, 20]         262,272
|    |    └─Mish: 3-11                   [-1, 128, 18, 20]   

In [ ]:
net = net.cuda()

In [ ]:
x = torch.randn(16, 1, 576, 640)

In [ ]:
x = x.cuda()

In [ ]:
xx, l = net(x)

In [ ]:
xx.shape

In [ ]:
l

In [ ]:
vq(x)[0].shape

In [ ]:
import os

In [ ]:
import glob

In [ ]:
from pathlib import Path

In [ ]:
list(Path(code_dir).glob("**/*.py"))

In [ ]:
for path in glob.glob(os.path.join(code_dir, "**/*.py"), recursive=True):
    print(path)

In [ ]:
e = Encoder(1, 64, 32, 0.2)

In [ ]:
d = Decoder(64, 1, 32, 0.2)

In [ ]:
z = e(x)

In [ ]:
xh = d(z)

In [ ]:
xh.shape

In [ ]:
a = [2, 2]

In [ ]:
a += [1, 1]

In [ ]:
a

In [ ]:
x

In [ ]:
576 // (2 * 4)

In [ ]:
16 // 2

In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [ ]:
# context initialization
with initialize(config_path="../training/conf/", job_name="test_app"):
    cfg = compose(config_name="config", overrides=["+experiment=vqvae"])
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

In [ ]:
mapping = instantiate(cfg.mapping)

In [ ]:
network = instantiate(cfg.network)

In [ ]:
x = torch.rand(1, 1, 576, 640)

In [ ]:
network.encode(x)[0].shape

In [ ]:
t, l = network(x)

In [ ]:
l

In [ ]:
from torch.nn import functional as F


In [ ]:
F.mse_loss(x, t) + l

In [ ]:
t.shape

In [ ]:
576 / 4

In [ ]:
t = torch.randint(0, 1006, (1, 451)).cuda()

In [ ]:
z = torch.rand((1, 36 * 40, 128)).cuda()

In [ ]:
network = network.cuda()

In [ ]:
network.decode(z, t).shape

In [ ]:
decoder = decoder.cuda()

In [ ]:
decoder(z, t).shape

In [ ]:
OmegaConf.set_struct(cfg, False)

In [ ]:
datamodule = instantiate(cfg.datamodule, mapping=mapping)

In [ ]:
datamodule.prepare_data()
datamodule.setup()

In [ ]:
len(datamodule.train_dataloader())

In [ ]:
mapping

In [ ]:
config = cfg

In [ ]:
loss_fn = instantiate(cfg.criterion)

In [ ]:
import hydra

In [ ]:
    model = hydra.utils.instantiate(
        config.model,
        mapping=mapping,
        network=network,
        loss_fn=loss_fn,
        optimizer_config=config.optimizer,
        lr_scheduler_config=config.lr_scheduler,
        _recursive_=False,
    )


In [ ]:
mapping.get_index

In [ ]:
net = instantiate(cfg)

In [ ]:
net

In [ ]:
img = torch.rand(4, 1, 576, 640)

In [ ]:
y = torch.randint(0, 1006, (4, 451))

In [ ]:
y.shape

In [ ]:
net = net.cuda()
img = img.cuda()
y = y.cuda()

In [ ]:
net(img, y).shape

In [ ]:
from torchsummary import summary

In [ ]:
summary(net, [(1, 576, 640), (451,)], device="cpu", depth=2)